In [32]:
import numpy as np

# 확률과정 

`-` 동전을 무한히 던지는 시행을 생각하자. 동전을 10번 던져서 결과를 관찰했다고 하자. 동전을 30번째 던져서 앞면이 나올지 뒷면이 나올지 알고 싶다면?

`-` 현재 삼성전자 주가는 66000이다. 20일뒤의 삼성전자 주가가 얼마일지 알고 싶다면? 

`-` 원래 미래를 예측하기 위해서 해야하는 과정 

![그림1: 1400만개의 미래를 탐색중인 Doctor Strange](https://i.redd.it/dkfg72px80y21.jpg)

`-` 하지만 현실적으로는 이게 너무 힘들지 않을까? 

# 날씨예측

`-` 아래와 같이 세상의 법칙이 있다고 하자. 

- 어제 맑음 $\to$ 오늘도 맑음: 40% // 오늘은 비: 60% 
- 어제 비 $\to$ 오늘은 맑음: 70% // 오늘도 비 30%

`-` 모든 $t$에 대하여 확률변수 $X_t$를 아래와 같이 정의하자. 

- $X_t=\begin{cases} 0 & \text{맑음} \\ 1 & \text{비} \end{cases}$ 


`-` 오늘 (2023년4월13일) 비가 왔다고 치자. 10000일 뒤에도 비가 올 확률은 얼마일까? 

# 풀이1

`-` $X_t=0$ 이면? (어제 비가 안왔으면?)

In [216]:
np.random.rand() < 0.6

True

`-` $X_t=1$ 이면? (어제 비가 왔으면?)

In [217]:
np.random.rand() < 0.3

False

`-` 두 코드를 합치면?

In [230]:
def rain(before):
    if before == True: # 비가왔으면 
        after = (np.random.rand() < 0.3)
    else: # 비가 안왔으면 
        after = (np.random.rand() < 0.6) 
    return after 

`-` 테스트 

In [231]:
sum([rain(0) for i in range(1000)])

620

In [232]:
sum([rain(1) for i in range(1000)])

294

`-` 하나의 $\omega$에 대응하는 확률과정 

In [233]:
def doctor_strange(today):
    lst = [today] 
    for i in range(9999): 
        lst.append(rain(lst[i]))
    return lst 

`-` 4305개의 $\omega$에 대응하는 확률과정 

In [239]:
today = True # 오늘은 비가 왔음 
arr = np.stack([doctor_strange(today) for i in range(4305)])

In [240]:
arr.shape

(4305, 10000)

In [241]:
arr[:,-1].mean()

0.4429732868757259

In [242]:
arr

array([[ True, False,  True, ..., False, False,  True],
       [ True, False,  True, ...,  True, False,  True],
       [ True, False,  True, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True,  True, False, ...,  True, False,  True],
       [ True,  True, False, ..., False,  True, False]])

# 풀이2 

`-` 세상의 법칙 

- $X_{t-1}=0 \Rightarrow X_{t}\overset{d}{=} Ber(0.6)$
- $X_{t-1}=1 \Rightarrow X_{t}\overset{d}{=} Ber(0.3)$

`-` 정리하면 

- $P(X_t=0) = P(X_{t-1}=0) \times 0.4  + P(X_{t-1}=1) \times 0.7$  
- $P(X_t=1) = P(X_{t-1}=0) \times 0.6  + P(X_{t-1}=1) \times 0.3$  

`-` 매트릭스형태로 바꾸면 

- $\begin{bmatrix} 
P(X_t=0)\\ 
P(X_t=1)
\end{bmatrix}= \begin{bmatrix} 0.4 & 0.7 \\ 0.6 & 0.3  \end{bmatrix} \begin{bmatrix} 
P(X_{t-1}=0)\\ 
P(X_{t-1}=1)
\end{bmatrix}$

- ${\boldsymbol \mu}_t = {\bf P} {\boldsymbol \mu}_{t-1}$

`-` 이렇게 놓고 보니까 아래를 관찰할 수 있다. 

- ${\boldsymbol \mu}_2 = {\bf P}{\boldsymbol \mu}_1$
- ${\boldsymbol \mu}_3 = {\bf P}{\boldsymbol \mu}_2 = {\bf P}^2{\boldsymbol \mu}_1$
- $\dots$
- ${\boldsymbol \mu}_n = {\bf P}^{n-1}{\boldsymbol \mu}_1$

`-` ${\bf P}^{999}$만 계산하면 끝나겠군? 

In [179]:
P = np.array([[0.4,0.7],[0.6,0.3]])
P

array([[0.4, 0.7],
       [0.6, 0.3]])

In [180]:
P@P # P의 제곱

array([[0.58, 0.49],
       [0.42, 0.51]])

In [181]:
P@P@P@P # P의 4제곱

array([[0.5422, 0.5341],
       [0.4578, 0.4659]])

In [182]:
P@P@P@P@P@P@P@P # P의 8제곱

array([[0.53849182, 0.53842621],
       [0.46150818, 0.46157379]])

In [183]:
P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P # P의 16제곱

array([[0.53846154, 0.53846154],
       [0.46153846, 0.46153846]])

근데 ${\bf P}$가 수렴하는거 같은데? 

In [184]:
P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P # P의 32제곱

array([[0.53846154, 0.53846154],
       [0.46153846, 0.46153846]])

대충 ${\bf P}^{9999} \approx {\bf P}^{32}$ 로 두어도 무방할 듯 

In [185]:
Plim = P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P@P

In [186]:
μ1 = np.array([[0],[1]])
μ1

array([[0],
       [1]])

In [187]:
Plim @ μ1

array([[0.53846154],
       [0.46153846]])

# 풀이3

`-` 세상의 법칙 

- $X_{t-1}=0 \Rightarrow X_{t}\overset{d}{=} Ber(0.6)$
- $X_{t-1}=1 \Rightarrow X_{t}\overset{d}{=} Ber(0.3)$

`-` 추측: 10000일 뒤에 비가 올 확률이 $p$라면 9999일 뒤에 비가 올 확률도 $p$일 것 같다. 

***이걸 가정하고 계산해보면***

`1` 9999일 뒤에 비가 안 올 확률 = $1-p$ 

- 9999일 뒤에 비가 오고 10000일 뒤에 비가 올 확률 = $0.6 (1-p)$ 
- 9999일 뒤에 비가 오고 10000일 뒤에 비가 안올 올 확률 = $0.4 (1-p)$ 

`2` 9999일 뒤에 비가 올 확률 = $p$ 

- 9999일 뒤에 비가 오고 10000일 뒤에 비가 올 확률 = $0.3 p$ 
- 9999일 뒤에 비가 오고 10000일 뒤에 비가 안올 올 확률 = $0.7 p$ 



따라서 $0.6(1-p) + 0.3p = p$ 이므로, 

- $0.6-0.3p = p$ 
- $p=6/13$ 

In [227]:
6/13

0.46153846153846156

# 풀이4 

In [238]:
np.mean(doctor_strange(True))

0.4629